In [1]:
# Initial imports
import os # Miscellaneous operating system interfaces (os.getenv)
import requests # requests.get
import pandas as pd
from dotenv import load_dotenv #Reads the key-value pair from .env file and adds them to environment variable.
from pathlib import Path
import json # lightweight data interchange format 
import matplotlib.pyplot as plt

%matplotlib inline
import FundamentalAnalysis as fa
import alpaca_trade_api as tradeapi


Bad key "text.kerning_factor" on line 4 in
C:\Users\Khemero\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Load .env enviroment variables
load_dotenv()
from pathlib import Path
env_path = Path('C:/Users/Khemero/.env')

In [3]:
# Set FA API key
api_key = os.getenv("FA2")

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
# Set the file path
sp_data = Path("Resources/sp.csv")
sp_symbols_data = Path("Resources/sp500_symbols.csv")
sp_ticker_data = Path("Resources/sp500_ticker.csv")

In [6]:
# Read whale returns csv
# Read the CSVs and set the `date` column as a datetime index to the DataFrame
sp_df = pd.read_csv(sp_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp_df.sort_index(ascending = True, inplace = True)

sp_symbols = pd.read_csv(sp_symbols_data, index_col="Symbol", infer_datetime_format=True, parse_dates=True)
sp_symbols.sort_index(ascending = True, inplace = True)

sp_ticker = pd.read_csv(sp_ticker_data, index_col="Symbol", infer_datetime_format=True, parse_dates=True)
sp_ticker.sort_index(ascending = True, inplace = True)

In [7]:
# Drop nulls
sp_df = sp_df.dropna()

In [8]:
# master_list = []
# for i in target_items:
#         data_dict = {}
#         data_dict['marketplace'] = 'ebay'
#         data_dict['date_retrieved'] = date.today()
#         data_dict['link'] = i.get('a',{'href'})
#         master_list.append(data_dict)


In [9]:
# Format current date as ISO format
today = pd.Timestamp("2020-07-31", tz="America/New_York").isoformat() 
start = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat() 
end = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat() # last year

# Set the tickers
tickers = ["TSLA", "NIO", "AAPL", "PFE", "MSFT", "MRNA", "ZM", "AMZN", "NNDM", "CRM", "NVDA"]
dict = {'stock':'', 'segment':''}  
marketcaps = {}
segments = {}
for stock in tickers:
    profile = fa.profile(stock, api_key) 
    try: 
        marketcaps[stock] = int(profile.iloc[22])
        segment = str(profile.iloc[26])[5:]
        segments[stock] = segment[:-28]
    except: continue
    

## Create the shares DataFrame with `tickers` as index
#df_shares = pd.DataFrame(shares_data, index=tickers)

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today
).df

In [10]:
# # Format current date as ISO format
# today = pd.Timestamp("2020-07-31", tz="America/New_York").isoformat() 
# start = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat() #1/1/2019?
# end = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat() #31/12/2019?

# # Set the tickers
# tickers = ["TSLA", "NIO", "AAPL", "PFE", "MSFT", "MRNA", "ZM", "AMZN", "NNDM", "CRM", "NVDA"]
# dict = {'stock':'', 'segment':''}  
# marketcaps = {}
# segments = {}
# for stock in tickers:
#     profile = fa.profile(stock, api_key) 
#     try: 
#         marketcaps[stock] = int(profile.iloc[22])
#         segment = str(profile.iloc[26])[5:]
#         segments[stock] = segment[:-28]
#     except: continue
    

# ## Create the shares DataFrame with `tickers` as index
# #df_shares = pd.DataFrame(shares_data, index=tickers)

# # Set timeframe to '1D' for Alpaca API
# timeframe = "1D"

# # Get current closing prices for SPY and AGG
# df_portfolio = alpaca.get_barset(
#     tickers,
#     timeframe,
#     start = today,
#     end = today
# ).df

In [11]:
marketcaps_df = pd.DataFrame(marketcaps.items(), columns=['stock', 'marketcap'])
segments_df = pd.DataFrame(segments.items(), columns=['stock', 'marketcap']).rename(columns={
    "marketcap":"sector"
})

In [12]:
marketcaps_df.head(11)

,stock,marketcap
0,TSLA,658791200000
1,NIO,72832279000
2,AAPL,2153362880000
3,PFE,209440506000
4,MSFT,1652649750000
5,MRNA,55490413000
6,ZM,116121706000
7,AMZN,1606431010000
8,NNDM,991197310
9,CRM,208098443000


In [13]:
segments_df.head(11)

,stock,sector
0,TSLA,Consumer Cyclical
1,NIO,Consumer Cyclical
2,AAPL,Technology
3,PFE,Healthcare
4,MSFT,Technology
5,MRNA,Healthcare
6,ZM,Communication Services
7,AMZN,Consumer Cyclical
8,NNDM,Technology
9,CRM,Technology


In [14]:
segments = segments_df.drop(columns=[
    "stock", 
])

In [15]:
joined_df = pd.concat([marketcaps_df, segments], axis="columns", join="inner")
joined_df.head(12)


,stock,marketcap,sector
0,TSLA,658791200000,Consumer Cyclical
1,NIO,72832279000,Consumer Cyclical
2,AAPL,2153362880000,Technology
3,PFE,209440506000,Healthcare
4,MSFT,1652649750000,Technology
5,MRNA,55490413000,Healthcare
6,ZM,116121706000,Communication Services
7,AMZN,1606431010000,Consumer Cyclical
8,NNDM,991197310,Technology
9,CRM,208098443000,Technology


In [17]:
joined_symbol_df = pd.concat([sp_symbols, sp_ticker], axis="columns", join="inner")
joined_symbol_df.head(500)

,#,Company,Weight,Price,Chg,% Chg,Security,GICS Sector,GICS Sub-Industry,Founded
Symbol,,,,,,,,,,
A,167,Agilent Technologies Inc.,0.119286,119.3,0.33,-0.28%,Agilent Technologies Inc,Health Care,Health Care Equipment,1999
AAL,446,American Airlines Group Inc.,0.027919,16.74,-0.06,(-0.36%),American Airlines Group,Industrials,Airlines,1934
AAP,417,Advance Auto Parts Inc.,0.035977,159.99,-0.35,(-0.22%),Advance Auto Parts,Consumer Discretionary,Automotive Retail,1932
AAPL,1,Apple Inc.,6.700995,127.3,-1.40,(-1.09%),Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",1977
ABBV,33,AbbVie Inc.,0.600080,104.45,-0.44,(-0.42%),AbbVie Inc.,Health Care,Pharmaceuticals,2013
ABC,368,AmerisourceBergen Corporation,0.047060,98.61,-0.13,(-0.13%),AmerisourceBergen Corp,Health Care,Health Care Distributors,1985
ABMD,392,ABIOMED Inc.,0.040985,293.39,11.40,-4.04%,ABIOMED Inc,Health Care,Health Care Equipment,1981
ABT,31,Abbott Laboratories,0.624418,108.97,0.19,-0.17%,Abbott Laboratories,Health Care,Health Care Equipment,1888
ACN,40,Accenture Plc Class A,0.545514,266.25,1.78,-0.67%,Accenture plc,Information Technology,IT Consulting & Other Services,1989


In [29]:
stock_data = fa.stock_data("TSLA", period="ytd", interval="1d")

In [31]:
sp_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2019-01-02 16:00:00,2476.96,2519.49,2467.47,2510.03,2233765369
2019-01-03 16:00:00,2491.92,2493.14,2443.96,2447.89,2754752741
2019-01-04 16:00:00,2474.33,2538.07,2474.33,2531.94,2661709794
2019-01-07 16:00:00,2535.61,2566.16,2524.56,2549.69,2526552370
2019-01-08 16:00:00,2568.11,2579.82,2547.56,2574.41,2458286256


In [30]:
stock_data

,open,close,low,high,volume,adjclose
2020-01-03,84.900002,86.052002,84.342003,86.139999,47660500,86.052002
2020-01-04,88.099998,88.601997,87.384003,90.800003,88892500,88.601997
2020-01-07,88.094002,90.307999,88.000000,90.311996,50665000,90.307999
2020-01-08,92.279999,93.811996,90.671997,94.325996,89410500,93.811996
2020-01-09,94.739998,98.428001,93.646004,99.697998,155721500,98.428001
2020-01-10,99.419998,96.267998,94.573997,99.760002,142202000,96.267998
2020-01-11,96.358002,95.629997,94.739998,96.987999,64797500,95.629997
2020-01-14,98.699997,104.972000,98.400002,105.125999,132588000,104.972000
2020-01-15,108.851997,107.584000,104.980003,109.482002,144981000,107.584000
2020-01-16,105.952003,103.699997,103.358002,107.568001,86844000,103.699997


In [27]:
growth_annually = fa.financial_statement_growth("TSLA", api_key, period="annual")

In [28]:
growth_annually

,2019,2018,2017,2016,2015
assetGrowth,0.153646,0.037837,0.264352,1.809153,0.383629
bookValueperShareGrowth,0.000000,0.000000,0.000000,0.000000,0.000000
debtGrowth,0.120924,0.160586,0.447014,1.643778,0.083724
dividendsperShareGrowth,0.000000,0.000000,0.000000,0.000000,0.000000
ebitgrowth,0.822198,0.762223,-1.445659,0.068779,-2.832241
epsdilutedGrowth,0.149195,0.516260,-1.528397,0.324846,0.410565
epsgrowth,0.149195,0.516260,-1.528397,0.324846,0.410565
fiveYDividendperShareGrowthPerShare,0.000000,0.000000,0.000000,0.000000,0.000000
fiveYNetIncomeGrowthPerShare,1.062972,8.237464,2.207711,0.849689,1.282897
fiveYOperatingCFGrowthPerShare,-30.532061,4.548054,-0.851095,-0.326564,0.623617


In [18]:
# # Collect general company information
# profile = fa.profile(ticker, api_key)

# # Collect recent company quotes
# quotes = fa.quote(ticker, api_key)

# # Collect market cap and enterprise value
# entreprise_value = fa.enterprise(ticker, api_key)

# # Show recommendations of Analysts
# ratings = fa.rating(ticker, api_key)

# # Obtain DCFs over time
# dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
# dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")

# # Collect the Balance Sheet statements
# balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")
# balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")

# # Collect the Income Statements
# income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
# income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")

# # Collect the Cash Flow Statements
# cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
# cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")

# # Show Key Metrics
# key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
# key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")

# # Show a large set of in-depth ratios
# financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
# financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")

# # Show the growth of the company
# growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
# growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

# # Download general stock data
# stock_data = fa.stock_data(ticker, period="ytd", interval="1d")

# # Download detailed stock data
# stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="2000-01-01", end="2020-01-01")
